In [3]:
import openpyxl

In [4]:
import pandas as pd

In [66]:
ro_df = pd.read_excel("Running Order Table Pocono Cup 2025.xlsx", engine="openpyxl")


ro_df = ro_df.rename(columns={"Lap/Position": "lap"})

In [151]:
df = pd.DataFrame(columns=("car_num", "lap_num", "lap_seconds", "start_feet"))

In [156]:
wb = openpyxl.load_workbook("Pocono Choose Line Export.xlsx", data_only=True)

sheetname = wb.sheetnames[0]

data = list(wb[sheetname].iter_rows(values_only=True))


extract_lap_numbers = []

# Set column/row start indexes
car_num_col = 0
lap_col_start = car_num_col + 3

# Skip the 1st row
raw_data = data[1:]

df_indx = 0

# Assume no rows are skipped
num_rows = len(raw_data)
indx = 0

process = {}
while indx < num_rows:

    # This ensures the data are collected
    if raw_data[indx][car_num_col] is not None:

        # Get the next two rows
        row = raw_data[indx:indx+3]

        car_num = int(row[0][car_num_col])

        lap_nums = []
        for xs in row[0][lap_col_start:]:
            
            if xs is None:
                break
            lap_nums.append(int(xs)) 

        
        result = {}

        lap_seconds = [float(xs) for xs in row[1][lap_col_start:lap_col_start+len(lap_nums)]]
        start_feet = [float(xs) for xs in row[2][lap_col_start:lap_col_start+len(lap_nums)]]
        
        for lap_num, seconds, feet in zip(lap_nums, lap_seconds, start_feet):
            
            df.loc[df_indx] = [car_num, lap_num, seconds, feet]

            print(car_num, lap_num)

            df_indx += 1

        process[car_num] = {"laps": lap_nums, "seconds": seconds, "feet": feet}

        indx += 3

1 0
1 0
1 1
1 2
1 3
1 4
1 5
1 6
1 7
1 8
1 9
1 10
1 11
1 12
1 13
1 14
1 15
1 16
1 17
1 18
1 19
1 20
1 21
1 22
1 23
1 24
1 25
1 26
1 27
1 28
1 29
1 30
1 31
1 32
1 33
1 34
1 35
1 36
1 37
1 38
1 39
1 40
1 41
1 42
1 43
1 44
1 45
1 46
1 47
1 48
1 49
1 50
1 51
1 52
1 53
1 54
1 55
1 56
1 57
1 58
1 59
1 60
1 61
1 62
1 63
1 64
1 65
1 66
1 67
1 68
1 69
1 70
1 71
1 72
1 73
1 74
1 75
1 76
1 77
1 78
1 79
1 80
1 81
1 82
1 83
1 84
1 85
1 86
1 87
1 88
1 89
1 90
1 91
1 92
1 93
1 94
1 95
1 96
1 97
1 98
1 99
1 100
1 101
1 102
1 103
1 104
1 105
1 106
1 107
1 108
1 109
1 110
1 111
1 112
1 113
1 114
1 115
1 116
1 117
1 118
1 119
1 120
1 121
1 122
1 123
1 124
1 125
1 127
1 128
1 129
1 130
1 131
1 132
1 134
1 136
1 137
1 138
1 139
1 140
1 141
1 143
1 144
1 145
1 146
1 147
1 148
1 149
1 150
1 152
1 153
1 154
1 155
1 156
1 157
1 158
1 159
2 0
2 0
2 1
2 2
2 3
2 4
2 5
2 6
2 7
2 8
2 9
2 10
2 11
2 12
2 13
2 14
2 15
2 16
2 17
2 18
2 19
2 20
2 21
2 22
2 23
2 24
2 25
2 26
2 27
2 28
2 29
2 30
2 31
2 32
2 33
2 34
2 35
2 

In [ ]:
df.car_num = df.car_num.astype(str)

In [551]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.mixture import GaussianMixture
from scipy import stats

In [552]:
def get_order_lap(lap):
    return ro_df[ro_df.lap.astype(int) == lap].drop("lap", axis=1).values.flatten()[:-5].astype(int).astype(str)

def get_start_lap(lap):
    return df[df.lap_num == lap]

In [269]:
def get_left_right_proba(x):
        # x = 1-D NumPy array of your samples ---------------------------
           # make sure it’s an array
        x = x.reshape(-1, 1)             # sklearn expects 2-D (n_samples, n_features)



        # 1. Fit the mixture
        gmm = GaussianMixture(
                n_components=2,           # two normals
                covariance_type="full",   # let each component have its own σ
                random_state=0)
        gmm.fit(x)

        means = gmm.means_.ravel()
        sigma  = np.sqrt(gmm.covariances_.ravel())
        print(sigma)
        r = gmm.predict_proba(x)

        if means[0] >= means[1]:
                r = np.concatenate([r[:, 1:2], r[:, 0:1]], axis=1)

                means = [means[1], means[0]]
                sigma = [sigma[1], sigma[0]]
          

        pdf_comp0 = stats.norm.pdf(x, means[0], sigma[0]).flatten()
        pdf_comp1 = stats.norm.pdf(x, means[1], sigma[1]).flatten()
        
        probs = np.array(list(zip(np.round(pdf_comp0, 2), np.round(pdf_comp1, 2))))

        return r, probs, 

In [270]:
def classify_restart(x):
    if x == 1:
        return "INNER"
    else:
        return "OUTER"


In [279]:
df['classification'] = None
df['confidence'] = None

In [453]:
import numpy as np

restart_laps = [35, 48, 60, 80, 87, 100, 130]
cutoff = 20
# for lap in ro_df.lap.values.flatten():
for lap in restart_laps:
    # print(lap)
    filter = (df.lap_num == lap)&(df.start_feet < 95)
    filter_next = (df.lap_num == lap+1)&(df.start_feet < 95)
    lap_df_filter = df[filter]
    next_lap_df_filter = df[filter_next]

    # Merge or match based on car_num and calculate weighted average
    matched_df = pd.merge(lap_df_filter[['car_num', 'start_feet']], next_lap_df_filter[['car_num', 'start_feet']], 
                        on='car_num', suffixes=('_current', '_next'))

    x = 0.05 * matched_df['start_feet_current'] + 0.95 * matched_df['start_feet_next']

    r, probs = get_left_right_proba(x.values)
    outer, inner = np.round(r[:, 0], 2), np.round(r[: 1], 2)

    classification = np.argmax(r, axis=1)

    output_proba = probs[np.arange(probs.shape[0]), classification]

    df.loc[filter, 'classification'] = classification
    df.loc[filter, 'confidence'] = output_proba
    
    df.loc[filter, 'restart_class'] = df.loc[filter, 'classification'].apply(lambda x: classify_restart(x))


    outer_cnt = (r>0.5).sum(axis=0)[0]
    inner_cnt = (r>0.5).sum(axis=0)[1]


[0.85763713 2.08363036]
[0.66466219 0.51540426]
[0.41456658 1.67490884]
[1.0127514  1.26075113]
[3.01674077 0.54080834]
[0.67348755 0.69050823]
[0.82273683 2.05713515]


In [562]:
def get_order_lap(lap):
    return ro_df[ro_df.lap.astype(int) == lap].drop("lap", axis=1).values.flatten()[:-7].astype(int).astype(str)

output = {}
for lap in restart_laps:
# for lap in [60]:
    old_seq = get_order_lap(lap-1)
    restart_seq = get_order_lap(lap)
    new_seq = get_order_lap(lap+1)

    old_pos = {id_: idx for idx, id_ in enumerate(old_seq)}

    restart_pos = {id_: idx for idx, id_ in enumerate(restart_seq)}

    new_pos = {id_: idx for idx, id_ in enumerate(new_seq)}

    tmp_df = pd.DataFrame(columns = ("old_positions", "restart_positions", "new_positions", "car_num")) 
    count= 0 
    for car in restart_pos:
        old_ = old_pos.get(car)
        restart_ = restart_pos.get(car)
        new_ = new_pos.get(car)

        tmp_df.loc[count] = [old_, restart_, new_, car]
        count+=1

    tmp = get_start_lap(lap)

    oof = tmp_df.merge(tmp, on=['car_num'], how='left')

    oof_2 = oof[['old_positions', "restart_positions", "new_positions", "car_num", "restart_class"]]

    inner_oof = oof_2[oof_2.restart_class == "INNER"][:16].reset_index(drop=True)
    outer_oof = oof_2[oof_2.restart_class == "OUTER"][:16].reset_index(drop=True)

    lap_order = []
    lap_result = []
    o = 0
    for indx in range(8):
        inner_car_num = inner_oof.loc[indx, "car_num"]
        outer_car_num = outer_oof.loc[indx, "car_num"]
        inner_order = inner_oof.loc[indx, "old_positions"]
        outer_order = outer_oof.loc[indx, "old_positions"]        
        # print(inner_car_num, outer_car_num)
        print(inner_order, outer_order)

        # potentially breaking

        if inner_order < outer_order:
            inner_order = o
            outer_order = o + 1
        else:
            inner_order = o + 1
            outer_order = o
            
        lap_order.append([inner_order, outer_order])
        lap_result.append([inner_car_num, outer_car_num])
        o += 2
    output[lap] = [lap_result, lap_order]
    print()

2 0
3 1
5 4
7 6
9 8
12 10
13 11
15 14

1 0
3 2
6 4
8 5
10 7
12 9
13 11
14 15

1 0
3 2
5 4
7 6
9 8
12 11
15 14
16 17

1 0
4 2
5 3
7 6
9 8
10 12
11 13
15 14

0 1
2 3
6 4
7 5
8 9
11 10
12 13
15 14

1 0
2 3
5 4
7 6
8 9
10 11
12 13
15 14

1 0
3 2
5 4
7 6
9 8
11 10
13 12
14 15



C:\Users\rohan\AppData\Local\Temp\ipykernel_58716\2536109768.py:2: RuntimeWarning: invalid value encountered in cast
  return ro_df[ro_df.lap.astype(int) == lap].drop("lap", axis=1).values.flatten()[:-7].astype(int).astype(str)
C:\Users\rohan\AppData\Local\Temp\ipykernel_58716\2536109768.py:2: RuntimeWarning: invalid value encountered in cast
  return ro_df[ro_df.lap.astype(int) == lap].drop("lap", axis=1).values.flatten()[:-7].astype(int).astype(str)
C:\Users\rohan\AppData\Local\Temp\ipykernel_58716\2536109768.py:2: RuntimeWarning: invalid value encountered in cast
  return ro_df[ro_df.lap.astype(int) == lap].drop("lap", axis=1).values.flatten()[:-7].astype(int).astype(str)
C:\Users\rohan\AppData\Local\Temp\ipykernel_58716\2536109768.py:2: RuntimeWarning: invalid value encountered in cast
  return ro_df[ro_df.lap.astype(int) == lap].drop("lap", axis=1).values.flatten()[:-7].astype(int).astype(str)
C:\Users\rohan\AppData\Local\Temp\ipykernel_58716\2536109768.py:2: RuntimeWarning: inval

In [563]:
from openpyxl import Workbook
from openpyxl.styles import Alignment, Font

def draw_restart_grid(ws, pairs, places, *, first_row=3):

    ws.column_dimensions["A"].width = 14
    ws.column_dimensions["B"].width = 3
    ws.column_dimensions["C"].width = 8
    ws.column_dimensions["D"].width = 8
    center = Alignment(horizontal="center", vertical="center")

    # {position → car_id}
    car_at = {}
    for (outer, inner), (pos_out, pos_in) in zip(pairs, places):
        car_at[pos_out] = outer
        car_at[pos_in] = inner
    # Only the first 16 positions are shown (0–15)
    running_rows = range(16)

    # Fill column A (Running Order) row-by-row
    for pos in running_rows:
        value = car_at.get(pos, "")          # blank if no car for that slot
        ws.cell(row=first_row + pos, column=1, value=value)

    # Draw the stacked OUTER / INNER blocks (still 8 pairs)
    for i, (outer, inner) in enumerate(pairs):
        top = first_row + i * 2
        bottom = top + 1

        for col_letter, car in (("C", outer), ("D", inner)):
            col_idx = ord(col_letter) - 64

            ws.merge_cells(
                start_row=top,    start_column=col_idx,
                end_row=bottom,   end_column=col_idx
            )

            cell = ws.cell(row=top, column=col_idx, value=car)
            cell.alignment = center
            cell.font = Font(bold=True)

wb = Workbook()
wb.remove(wb.active)

for lap, (pairs, places) in output.items():
    ws = wb.create_sheet(f"Lap {lap}")

    # sheet title (row 1)
    ws.merge_cells(start_row=1, end_row=1, start_column=1, end_column=4)
    title = ws["A1"]
    title.value = f"Lap {lap}"
    title.alignment = Alignment(horizontal="center")
    title.font = Font(bold=True, size=14)

    # headings (row 2)
    ws["A2"].value = "Running Order"
    ws["A2"].alignment = Alignment(horizontal="center")
    ws["A2"].font = Font(bold=True)

    ws.merge_cells(start_row=2, end_row=2, start_column=3, end_column=4)
    ws["C2"].value = "Line Up"
    ws["C2"].alignment = Alignment(horizontal="center")
    ws["C2"].font = Font(bold=True)

    # draw the grid (starting row 3)
    draw_restart_grid(ws, pairs, places, first_row=3)

wb.save("Rohan_PoconoTest.xlsx")